In [24]:
%run "..\general_functions\generalFunctions.ipynb"

#### Slide 1:MixAnalysis

In [ ]:
def Mixanalysis(prs,dataframe,numOfDuplicates,Inscop='Sector',position=0,label_col=''):
    for slidenum in range(numOfDuplicates):
        market=list(dataframe.keys())[slidenum]
        df=dataframe[market].copy()
        shapes = prs.slides[slidenum+position].shapes
        charts = []
        tables = []
        title = shapes.title.text
        
        headerNumber = get_shape_number(shapes, "Mix Analysis (Replace with So What)")

        titleNumber = headerNumber-1
        shapes[titleNumber-1].text = data_source
        shapes[titleNumber].text ='Net Sales, NS/Vol, Gross Margin | By '+ Inscop +' | '+ market + ' | P12M''\n''Bubble Size: Net Sales, Bubble Colour: Value IYA'
        shapes[titleNumber].text_frame.paragraphs[0].font.bold = False

        for shape in shapes:
            if shape.has_chart:
                shape_id = shape.shape_id
                charts.append(shape)
        chart = charts[0].chart
    
        # charts[0].left = Inches(0.57)
        chart_name = charts[0].name
        chart_type = chart.chart_type
        chart_data = BubbleChartData()
        chart_data.categories = df['Net Sales/Kg'].unique().tolist()
        series = chart_data.add_series('Gross Margin %')
        series.has_data_labels = True
        for i in range(df.shape[0]):
            series.add_data_point(df['Net Sales/Kg'].iloc[i], df['Gross Margin %'].iloc[i], df['Net Sales'].iloc[i])
        chart.replace_data(chart_data)
        xlsx_file=BytesIO()
        with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
            chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
            worksheet.write(0, 4, "labels")
            worksheet.write_column(1, 4, df[label_col], None)

        chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

        category_axis = chart.category_axis
        category_axis.auto_axis = True   
        category_axis=chart.category_axis
        category_axis.minimum_scale=0
        category_axis.maximum_scale=round(df["Net Sales/Kg"].max()+1)*1.2
    
        currencywithoutspace =currency.strip()  # Remove the leading space
        category_axis.axis_title.text_frame.text = f"Net Sales/ Vol ({currencywithoutspace})"  # Set the axis title text
        category_axis.axis_title.text_frame.paragraphs[0].font.size = Pt(8)
        category_axis.axis_title.text_frame.paragraphs[0].font.name = 'Nexa Bold'
        category_axis.has_number_format = True

        if sign.lower() == 'before':
            category_axis.tick_labels.number_format = f'#,##0.00'  if decimals == 2 else f'#,##0'
        else:
            category_axis.tick_labels.number_format = f'#,##0.00'  if decimals == 2 else f'#,##0'

        value_axis = chart.value_axis
        value_axis.minimum_scale = (df["Gross Margin %"].min()-0.3)*0.8
        value_axis.maximum_scale=(df["Gross Margin %"].max()+0.3)*1.2
        value_axis.tick_labels.number_format = '0%'
        value_axis.auto_axis = True

        df['Weight'] = df['Net Sales'] / sum(df['Net Sales'])

        df['Gross Margin % Weighted'] = df['Gross Margin %'] * df['Weight']
        Gross_Weight_value = sum(df['Gross Margin % Weighted']) / sum(df['Weight'])

        vertical_cross_value = Gross_Weight_value  # Set the cross point based on the average Gross Margin %
        value_axis.crosses = XL_AXIS_CROSSES.CUSTOM
        value_axis.crosses_at = vertical_cross_value
        # calculate the Weighted values 
        df['Net Sales/Kg Weighted'] = df['Net Sales/Kg'] * df['Weight']
        Net_Sales_Weight_value = sum(df['Net Sales/Kg Weighted']) / sum(df['Weight'])

        horizontal_cross_value = Net_Sales_Weight_value
        category_axis.crosses = XL_AXIS_CROSSES.CUSTOM
        category_axis.crosses_at = horizontal_cross_value
       

        chart_top = charts[0].top
        chart_height = charts[0].height
        chart_width = charts[0].width
        chart_down=chart_top + chart_height
        chart_left=charts[0].left
        chart_right=chart_left + chart_width

        proportion=vertical_cross_value / value_axis.maximum_scale
# Get axis scaling details
        vertical_min_scale = value_axis.minimum_scale
        vertical_max_scale = value_axis.maximum_scale
 
    # Calculate the proportion of the vertical_cross_value within the axis range
        vertical_value_range = vertical_max_scale - vertical_min_scale
        vertical_value_proportion = (vertical_cross_value - vertical_min_scale) / vertical_value_range
    # Calculate the position of the vertical_cross_value in chart coordinates
        chart_y_position = (1 - vertical_value_proportion) * chart_height  # Invert because Y axis increases downwards
    # Convert chart y position to slide position in inches
        slide_vertical_position = (chart_top + chart_y_position)*0.975
        min_scale = category_axis.minimum_scale
        max_scale = category_axis.maximum_scale
        value_range = max_scale - min_scale
        horizontal_proportion = (horizontal_cross_value - min_scale) / value_range
        chart__position=  horizontal_proportion* chart_width
        slide_horizontal_position = (chart_left + chart__position)*0.975

        if pd.isna(slide_vertical_position):
            slide_vertical_position = 0  # or any other default value
        if pd.isna(slide_horizontal_position):
            slide_horizontal_position = 0  # or any other default value

        # Set the positions of the shapes based on calculated values (converting from EMUs to inches)
        shapes[8].top = Inches(slide_vertical_position/ 914400 )
       
        shapes[15].left = Inches(slide_horizontal_position / 914400)

        for i,point in enumerate(chart.series[0].points):

            point.format.fill.solid()
            
            value_sales_iya = df['Value Sales IYA'].iloc[i]

            # if pd.isnull(value_sales_iya):  # Check if value_sales_iya is NaN
            #     value_sales_iya = 0  # Replace NaN with 0
            value_sales_iya *= 100  
            # print(value_sales_iya)
            if (value_sales_iya == -19900.0):
                point.format.fill.fore_color.rgb = RGBColor(174, 171, 171)  # Light Grey
            elif value_sales_iya < 90:
                point.format.fill.fore_color.rgb = RGBColor(255, 128, 128)  # Light Red
                # print(f"Color set to: {point.format.fill.fore_color.rgb} for {df[label_col].iloc[i]}")

            elif 90 <= value_sales_iya < 98:
                point.format.fill.fore_color.rgb = RGBColor(255, 191, 191)  # Light Pink

            elif (98 <= value_sales_iya < 102):
                point.format.fill.fore_color.rgb = RGBColor(230, 229, 229)  # Light Grey

            elif 102 <= value_sales_iya < 110:
                point.format.fill.fore_color.rgb = RGBColor(178, 223, 220)  # Light Blue
            else:
                point.format.fill.fore_color.rgb = RGBColor(72, 174, 166)  # Dark Blue



#### Slide 2:TradeMargin 

In [ ]:
def TradeMargin(prs, dataframe, numOfDuplicates, position=0,InScope=" "):
    # Loop through each slide number
    for slidenum in range(numOfDuplicates):
        # Get market from dataframe keys
        market = list(dataframe.keys())[slidenum]
       
        # Retrieve DataFrame for the current market
        df = dataframe[market].reset_index(drop=True)
        # Update title
        shapes = prs.slides[slidenum + position].shapes
         
        headerNumber = get_shape_number(shapes, "Trade Margin Analysis by Retailer (Replace with So What)")
        titleNumber = get_shape_number(shapes, "Trade Margin Analysis by Sector | Chocolate | Panda | Hershey’s | P12M")
       
        shapes[titleNumber-1].text = data_source
        shapes[titleNumber].text = f'Trade Margin Analysis by {InScope} | {market} | P12M'
 
        # Create table and chart
        tables, charts = createTableAndChart(shapes)
        table = tables[0].table
       
        # Remove unnecessary rows
        num_rows_to_remove = len(table.rows) - df[InScope].nunique() - 1
        table_height = get_table_height(table)
        for _ in range(num_rows_to_remove):
            if len(table.rows) > 1:
                row = table.rows[1]
                remove_row(table, row)
        # Adjust row heights
        total_row_height = table_height - table.rows[0].height
        num_rows = len(table.rows) - 1
        if num_rows > 0:
            cell_height = total_row_height / num_rows
            for row in range(1, len(table.rows)):
                table.rows[row].height = int(cell_height)
       
        # Populate table cells
        for i, row in enumerate(table.rows):
            if i == 0:  # Skip header row
                continue
            for j, cell in enumerate(row.cells):
                if j == 0:  # InScope column
                    cell.text = df[InScope].iloc[i - 1]
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                elif j == 2:  # Value Sales IYA column
                    cell.text = str(int(round(df['Value Sales IYA'].replace(np.nan, 0).iloc[i - 1] * 100, 0)))
                elif j == 5:  # Trade Profit IYA column
                    cell.text = str(int(round(df['Trade Profit IYA'].replace(np.nan, 0).iloc[i - 1] * 100, 0)))
                elif j == 6:  # VSOD IYA column
                    cell.text = str(int(round(df['VSOD IYA'].replace(np.nan, 0).iloc[i - 1] * 100, 0)))
               
                cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                cell.text_frame.paragraphs[0].font.size = Pt(8)
                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
       
 
 
        df['Value Sales'] = df['Value Sales'].apply(lambda x: int(round(x / percent)))
        filtered_dataframe= df[df[InScope] != 'Total']
        filtered_dataframe['Trade_Margin_DYA'] = (filtered_dataframe['Trade Margin % DYA'] ).replace(np.nan,0)
        # print(filtered_dataframe, market)
 
 
        total_row_values = df[df[InScope] == 'Total']
        total_row_values['Trade_Margin_DYA'] = (total_row_values['Trade Margin % DYA'] ).replace(np.nan,0)
        # print(total_row_values, InScope)
        total_value_sales = total_row_values['Value Sales'].iloc[0]  # Get the 'Value Sales' for the 'Total' row
        total_value_sales_iya=round(total_row_values['Value Sales IYA'].iloc[0]*100)
        total_trade_margin=round(total_row_values['Trade Margin %'].iloc[0]*100)
        total_trade_margin_dya=round(total_row_values['Trade_Margin_DYA'].iloc[0]*100)
        # print(total_trade_margin_dya)
        total_trade_profit=round(total_row_values['Trade Profit IYA'].iloc[0]*100)
        total_VSOD=round(total_row_values['VSOD IYA'].iloc[0]*100)


        if table.rows !=0:
            last_row_index = len(table.rows) - 1  # Index of the last row
            last_row = table.rows[last_row_index]
            last_row.cells[0].text = 'Total'
            if sign.lower() == 'before':
                last_row.cells[1].text = f'{currency}{total_value_sales:,.0f}'
            else:
                last_row.cells[1].text = f'{total_value_sales:,.0f}{currency}'

            last_row.cells[2].text = str(total_value_sales_iya)
            last_row.cells[3].text = f'{total_trade_margin}'+"%"
            last_row.cells[4].text = f'{total_trade_margin_dya}'+"%"
            last_row.cells[5].text = str(total_trade_profit)
            last_row.cells[6].text = str(total_VSOD)
            for cell in last_row.cells:
                cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                cell.text_frame.paragraphs[0].font.size = Pt(8)
                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
 
 
        empty_row = {col: None for col in filtered_dataframe.columns}
        empty_row_df = pd.DataFrame([empty_row])
        filtered_dataframe = pd.concat([filtered_dataframe, empty_row_df], ignore_index=True)
        filtered_dataframe =filtered_dataframe.replace(np.nan, "")

        # Configure charts
        for chartNum in [0, 1, 2]:
            chart = charts[chartNum].chart
            chart_data = CategoryChartData()
            chart_data.categories = filtered_dataframe[InScope].tolist()

            if chartNum == 0:
                chart_data.add_series('Trade Margin DYA', filtered_dataframe['Trade_Margin_DYA'])

                # Format data labels to show + for positives
                data_labels = chart.plots[0].series[0].data_labels
                data_labels.number_format = '+0%;-0%;0%'

             # Update chart data and axis scaling
            elif chartNum == 1:
                t=chart_data.add_series('Trade Margin %', filtered_dataframe['Trade Margin %'])
            elif chartNum == 2:
                chart_data.add_series('Value Sales',filtered_dataframe['Value Sales'] )
 
                data_labels = chart.plots[0].series[0].data_labels
                # data_labels.number_format = f'#,##0{currency}'
               
 
                if sign.lower() == 'before':
                    data_labels.number_format = f'"{currency}"#,##0'
                else:
                    data_labels.number_format = f'#,##0"{currency}"'
 
 
            chart.series[0].invert_if_negative = True
            chart.replace_data(chart_data)

#### Slide 3 : SectorKPI

In [ ]:
def sectorKPI(prs,merged_data, duplication, position, Scope ="Sector"):
    slide_num =0
    for key, df in merged_data.items():
        slide = prs.slides[slide_num + position]  
        df['P3M Sales Rate Ix'] = (df['Rate of Sales_P3M'] / df['Rate of Sales_P3M'][0])*100 
        
        shapes = slide.shapes
        titleNumber = get_shape_number(shapes, "Sector KPIs Summary | Cream Cheese | National | Hershey")
        headereNumber = get_shape_number(shapes, "Sector KPIs Summary (Replace with So What)")
        shapes[titleNumber-1].text = data_source
        shapes[titleNumber].text = shapes[titleNumber].text.replace('Sector',Scope).replace('Cream Cheese', key.split(' | ')[0]).replace('National',key.split(' | ')[2]).replace("Hershey",key.split(' | ')[1])
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
        shapes[headereNumber].text = shapes[headereNumber].text.replace('Sector',Scope)
        shapes[headereNumber].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[headereNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'

        # Create and retrieve table and chart shapes
        tables, charts = createTableAndChart(slide.shapes)
        table = tables[0].table

        num_rows_to_remove = len(table.rows) - df.shape[0] - 1
        #table_height = 3.84
        table = removeRowFromTable(table, num_rows_to_remove, rowToExclude=1)
        # Populate the table with promotion data
        for row_number, row in enumerate(table.rows, start=0):
            for column_num, cell in enumerate(row.cells):
                if column_num == 2 and row_number ==0:
                    cell.text = "P12M\n Net Sales (" +percentstr +")"
                    set_cell_font(cell, 'Nexa Bold', 9)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[1].alignment = PP_ALIGN.CENTER
                    
                elif column_num ==0 and row_number >0:
                    cell.text = str(row_number)
                    set_cell_font(cell, 'Nexa Book', 8)
                elif column_num ==1 and row_number ==0:
                    cell.text = Scope
                    set_cell_font(cell, 'Nexa Bold', 9)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                elif column_num ==1 and row_number >0:
                    value = df[Scope][row_number-1]
                    cell.text = value
                    set_cell_font(cell, 'Nexa Book', 8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.LEFT
                elif column_num ==2 and row_number >0:
                    value = format_number(df['Net Sales_P12M'][row_number-1] / percent,use_decimals=True, decimals=1)
                    cell.text = str(value)
                    set_cell_font(cell, 'Nexa Book', 8)
                elif column_num ==3 and row_number >0:
                    value = format_number(df['Gross Margin %_P12M'][row_number-1] * 100,use_decimals=True, decimals=0)
                    cell.text = str(value) + "%"
                    set_cell_font(cell, 'Nexa Book', 8)
                elif column_num ==4 and row_number >0:
                    value = format_number(df['WD_P3M'][row_number-1]*100,use_decimals=True, decimals=0)
                    cell.text = str(value) + "%"
                    set_cell_font(cell, 'Nexa Book', 8)
                elif column_num ==5 and row_number >0:
                    value = format_number(df['Trade Margin %_P12M'][row_number-1]*100,use_decimals=True, decimals=0)
                    cell.text = str(value) + "%"
                    set_cell_font(cell, 'Nexa Book', 8)
                elif column_num ==6 and row_number >0:
                    value = format_number(df['P3M Sales Rate Ix'][row_number-1],use_decimals=True,decimals =0)
                    cell.text = str(value)
                    set_cell_font(cell, 'Nexa Book', 8)
                
        slide_num +=1        


#### Slide 4 : SkuKpis

In [28]:
def SkuKpis(prs, dataframe, numOfDuplicates, position=0):
    # Loop through each slide number
    for slidenum in range(numOfDuplicates):
        # Get market from dataframe keys
        market = list(dataframe.keys())[slidenum]
        
        # Retrieve DataFrame for the current market
        df= dataframe[market].reset_index(drop=True)
        # Update title
        shapes = prs.slides[slidenum + position].shapes


        headerNumber = get_shape_number(shapes, "SKU KPIs Summary (Replace with So What)")
        titleNumber = headerNumber-1
        shapes[titleNumber].text = f'SKU KPIs Summary | {market}'
        shapes[titleNumber-1].text = data_source


        # Create table and chart
        tables, charts = createTableAndChart(shapes)
        table = tables[0].table
        
        # Remove unnecessary rows
        num_rows_to_remove = len(table.rows) - df[f"{prodORitem}"].nunique() - 1
        table_height = get_table_height(table)
        for _ in range(num_rows_to_remove):
            if len(table.rows) > 1:
                row = table.rows[1]
                remove_row(table, row)
        # Adjust row heights
        total_row_height = table_height - table.rows[0].height
        num_rows = len(table.rows) - 1
        if num_rows > 0:
            cell_height = total_row_height / num_rows
            for row in range(1, len(table.rows)):
                table.rows[row].height = int(cell_height)
        
        # Populate table cells
        for i, row in enumerate(table.rows):
            if i == 0:  # Skip header row
                continue
            for j, cell in enumerate(row.cells):
                if j == 0:  # Index column (1 to length of Product)
                    cell.text = str(i)  # Assuming you want 1-based index
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                elif j == 1 and f'{prodORitem}' in df.columns:  # Product column
                    cell.text = df[f"{prodORitem}"].iloc[i - 1]
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                elif j == 2 and 'Net Sales_P12M' in df.columns:  # Net Sales_12M column
                    cell.text = str(round(df['Net Sales_P12M'].replace(np.nan, 0).iloc[i - 1]/percent, 1))
                elif j == 3 and 'Gross Margin %' in df.columns:  # Gross Margin % column
                    cell.text = str(int(round(df['Gross Margin %'].replace(np.nan, 0).iloc[i - 1] * 100, 0)))+"%"
                elif j == 4 and 'WD_P3M' in df.columns:  # WD_3M column
                    cell.text = str(int(round(df['WD_P3M'].replace(np.nan, 0).iloc[i - 1] * 100, 0)))+"%"
                elif j == 5 and 'Trade Margin %' in df.columns:  # Trade Margin % column
                    cell.text = str(int(round(df['Trade Margin %'].replace(np.nan, 0).iloc[i - 1] * 100, 0)))+"%"
                elif j == 6 and 'Sales Rate Ix' in df.columns:  #  Sales Rate Ix column
                    cell.text = str(int(round(df['Sales Rate Ix'].replace(np.nan, 0).iloc[i - 1] * 100, 0)))
                cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                

                cell.text_frame.paragraphs[0].font.size = Pt(8)
                cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

#### Slide 5: MixMatrix

In [1]:
# def mixMatrix(prs,dfs,position = 0,topbrans=False):
    
#     for slidenum,key in enumerate(dfs.keys()):
#         shapes = prs.slides[slidenum+position].shapes
#         tables, charts = createTableAndChart(shapes)
        
#         title_num = get_shape_number(shapes,"Mix Matrix | Weight of Business (WoB) vs. Net Sales per kg (NS/kg) and Gross Margin (GM%) | Client | P12M")
#         data_source_num = get_shape_number(shapes,"Data Source l Client P&L")
#         shapes[data_source_num].text = data_source
#         if topbrans==False:
#             shapes[title_num].text = shapes[title_num].text.replace('Client',key.split(' | ')[1])
#         else:
#             shapes[title_num].text = shapes[title_num].text.replace('Client',client_manuf[0])
   
#         shapes[title_num].text_frame.paragraphs[0].font.size = Pt(12)
#         shapes[title_num].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'

#         df = dfs[key].reset_index(drop=True)
#         col = df.columns[0]
#         table = tables[0].table
        
#         num_rows_to_remove = int(len(table.rows) - ((df.shape[0]/df.Source.nunique())*2 + 1))
#         table = removeRowFromTable(table,num_rows_to_remove,rowToExclude=1,height=Inches(3.88))

#         num_cols_to_remove = int(len(table.columns) - ((df.Source.nunique())*2 + 1))
#         table = col_cell_remove(table, num_cols_to_remove)

#         for row_num, row in enumerate(table.rows):

#             for col_num, cell in enumerate(row.cells):

#                 if row_num==0:
#                     if col_num % 2!=0:
#                         cell.text=df.Source.unique()[col_num//2]
#                         cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                         cell.text_frame.paragraphs[0].font.size = Pt(10)
#                         cell.text_frame.paragraphs[0].font.bold = True
#                         cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87,85,85)
#                         cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'

#                 elif row_num %2 !=0:
#                     if col_num == 0:
#                         cell.text=df[col].unique()[row_num//2].replace('Grand Total',col + ' Total')
#                         cell.text_frame.paragraphs[0].alignment = PP_ALIGN.LEFT
#                         cell.text_frame.paragraphs[0].font.size = Pt(10)
#                         cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
#                         cell.text_frame.paragraphs[0].font.bold = False

#                     elif col_num%2 !=0:
#                         df1=df[df.Source ==df.Source.unique()[col_num//2]].reset_index(drop=True)
#                         cell.text = 'Wob = '+str(int(round(df1['Total WoB %'].iloc[(row_num//2)]*100,0)))+'%'

#                         cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
#                         cell.text_frame.paragraphs[0].font.size = Pt(8)
#                         cell.text_frame.paragraphs[0].font.bold = False
#                         cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'

#                         if (col_num!= 1) & (row_num!=1):
#                             cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
#                             cell.text_frame.paragraphs[0].font.bold = True


#                 else:
#                     if col_num!=0:

#                         if col_num%2!=0:
#                             df1=df[df.Source ==df.Source.unique()[col_num//2]].reset_index(drop=True)

#                             value =format_number(df1['Net Sales/Kg'].iloc[(row_num//2)-1], use_decimals=True, decimals=decimals, use_apostrophes=False, currency_symbol=currency.strip(), currency_before=True if sign.lower=='before' else False)
#                             value = '' if float(value.split(' ')[0])==0 else value

#                             cell.text = 'NS/KG'+'\n'+str(value)
#                             div = (df1['Net Sales/Kg'].iloc[(row_num//2)-1]/df1['Net Total'].replace(0,1).iloc[0])*100
#                             cell.fill.fore_color.rgb= RGBColor(255,191,191) if div<80 else RGBColor(203,234,231) if div >120 else RGBColor(242,242,242)


#                         else:
#                             cell.text = 'GM%'+'\n'+str(int(round(df1['Gross Margin %'].iloc[(row_num//2)-1]*100,0)))+'%'
#                             div = (df1['Gross Margin %'].iloc[(row_num//2)-1]/df1['GM Total'].replace(0,1).iloc[0])*100
#                             cell.fill.fore_color.rgb= RGBColor(255,191,191) if div<80 else RGBColor(203,234,231) if div >120 else RGBColor(242,242,242)

#                         style_name='Nexa Book (Body)' #if (col_num not in [1,2]) & (row_num!=2) else 'Nexa Bold'
#                         for cell_style in cell.text_frame.paragraphs:
#                             cell_style.alignment = PP_ALIGN.CENTER
#                             cell_style.font.size = Pt(8)
#                             cell_style.font.bold = False
#                             cell_style.font.name = style_name


In [ ]:
def mixMatrix(prs,dfs,position = 0,topbrans=False,slideby=""):
    
    for slidenum,key in enumerate(dfs.keys()):
        shapes = prs.slides[slidenum+position].shapes
        tables, charts = createTableAndChart(shapes)
        if slideby=="bybrand":
            title_num = get_shape_number(shapes,"Mix Matrix | Weight of Business (WoB) vs. Net Sales per Vol (NS/Vol) and Gross Margin (GM%) | National | P12M")
            data_source_num = get_shape_number(shapes,"Data Source l Client P&L")
            shapes[4].text = data_source
            shapes[5].text = shapes[5].text.replace('National',key)

        else:
            title_num = get_shape_number(shapes,"Mix Matrix | Weight of Business (WoB) vs. Net Sales per Vol (NS/Vol) and Gross Margin (GM%) | Client | P12M")
            data_source_num = get_shape_number(shapes,"Data Source l Client P&L")
            shapes[4].text = data_source
            
            if topbrans==False:
                shapes[5].text = shapes[5].text.replace('Client',key.split(' | ')[1])
            else:
                shapes[5].text = shapes[5].text.replace('Client',client_manuf[0])
    
        shapes[5].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[5].text_frame.paragraphs[0].font.name = 'Nexa (Headings)'

        df = dfs[key].reset_index(drop=True)
        col = df.columns[0]
        table = tables[0].table
        
        num_rows_to_remove = int(len(table.rows) - ((df.shape[0]/df.Source.nunique())*2 + 1))
        table = removeRowFromTable(table,num_rows_to_remove,rowToExclude=1,height=Inches(3.88))

        num_cols_to_remove = int(len(table.columns) - ((df.Source.nunique())*2 + 1))
        table = col_cell_remove(table, num_cols_to_remove)

        for row_num, row in enumerate(table.rows):

            for col_num, cell in enumerate(row.cells):

                if row_num==0:
                    if col_num % 2!=0:
                        cell.text=df.Source.unique()[col_num//2]
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].font.size = Pt(10)
                        cell.text_frame.paragraphs[0].font.bold = True
                        cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(87,85,85)
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'

                elif row_num %2 !=0:
                    if col_num == 0:
                        cell.text=df[col].unique()[row_num//2].replace('Grand Total',col + ' Total')
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.LEFT
                        cell.text_frame.paragraphs[0].font.size = Pt(10)
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                        cell.text_frame.paragraphs[0].font.bold = False

                    elif col_num%2 !=0:
                        df1=df[df.Source ==df.Source.unique()[col_num//2]].reset_index(drop=True)
                        cell.text = 'Wob = '+str(int(round(df1['Total WoB %'].iloc[(row_num//2)]*100,0)))+'%'

                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].font.size = Pt(8)
                        cell.text_frame.paragraphs[0].font.bold = False
                        cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'

                        if (col_num!= 1) & (row_num!=1):
                            cell.text_frame.paragraphs[0].font.name = 'Nexa Bold'
                            cell.text_frame.paragraphs[0].font.bold = True


                else:
                    if col_num!=0:

                        if col_num%2!=0:
                            df1=df[df.Source ==df.Source.unique()[col_num//2]].reset_index(drop=True)

                            value =format_number(df1['Net Sales/Kg'].iloc[(row_num//2)-1], use_decimals=True, decimals=decimals, use_apostrophes=False, currency_symbol=currency.strip(), currency_before=True if sign.lower=='before' else False)
                            value = '' if float(value.split(' ')[0])==0 else value

                            cell.text = 'NS/Vol'+'\n'+str(value)
                            div = (df1['Net Sales/Kg'].iloc[(row_num//2)-1]/df1['Net Total'].replace(0,1).iloc[0])*100
                            cell.fill.fore_color.rgb= RGBColor(255,191,191) if div<80 else RGBColor(203,234,231) if div >120 else RGBColor(242,242,242)


                        else:
                            cell.text = 'GM%'+'\n'+str(int(round(df1['Gross Margin %'].iloc[(row_num//2)-1]*100,0)))+'%'
                            div = (df1['Gross Margin %'].iloc[(row_num//2)-1]/df1['GM Total'].replace(0,1).iloc[0])*100
                            cell.fill.fore_color.rgb= RGBColor(255,191,191) if div<80 else RGBColor(203,234,231) if div >120 else RGBColor(242,242,242)

                        style_name='Nexa Book (Body)' #if (col_num not in [1,2]) & (row_num!=2) else 'Nexa Bold'
                        for cell_style in cell.text_frame.paragraphs:
                            cell_style.alignment = PP_ALIGN.CENTER
                            cell_style.font.size = Pt(8)
                            cell_style.font.bold = False
                            cell_style.font.name = style_name

#### Slide 6:SectorSpendingPool

In [ ]:
def sectorSpendingPool(prs, data,duplication, position, Scope ="Sector"):
    #Fixed
    slide_num =0
    for key, df in data.items():
        df = df[~df[Scope].str.contains('Total', case=False)].reset_index(drop=True) 
        slide = prs.slides[slide_num+position]
        tables, charts = createTableAndChart(slide.shapes)
        table = tables[0].table
        chart = charts[0].chart
        shapes = slide.shapes
        titleNumber = get_shape_number(shapes, "Sector Spending pool split | Chocolate | Panda | Hershey | P12M")
        print(titleNumber)
        headereNumber = get_shape_number(shapes, "Sector Spending pool split (Replace with So What)")
        sourceNumber = get_shape_number(shapes, "Data Source l Client P&L")
        shapes[sourceNumber].text = data_source
        shapes[titleNumber].text = shapes[titleNumber].text.replace('Sector', Scope).replace('Chocolate', key.split(' | ')[0]).replace('Panda',key.split(' | ')[1]).replace("Hershey",key.split(' | ')[2])
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
        shapes[headereNumber].text =shapes[headereNumber].text.replace("Sector", Scope)
        shapes[headereNumber].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[headereNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'


        categories = df['Av Price/KG'].tolist()
        labeled_categories = [f"Av Price/Vol {format_number(price,use_decimals=True,decimals=2)}" for price in categories]
        chart_data = CategoryChartData()
        chart_data.categories = labeled_categories
        chart_data.add_series('GP', df['Gross Profit/Kg'])
        chart_data.add_series('COGS', df['Total COGS/Kg']*-1)
        chart_data.add_series('TP', df['Trade Profit/Kg'])
        chart_data.add_series('VAT', df['VAT/Kg'])

        chart.replace_data(chart_data)
        num_columns_to_remove = len(table.columns) - df[Scope].nunique() -1
        table = col_cell_remove(table, num_columns_to_remove)
        for row_number, row in enumerate(table.rows):
            for column_number, cell in enumerate(row.cells):
                if row_number ==0 and column_number >0:
                    value = df[Scope][column_number-1]
                    cell.text = value
                    set_cell_font(cell, 'Nexa Bold', 8)

                elif row_number == 1 and column_number >0:
                    value = df['Trade Profit/Kg'][column_number-1] / df['Gross Profit/Kg'][column_number-1]
                    cell.text= str(format_number(value,use_decimals= True,decimals=1))
                    set_cell_font(cell, 'Nexa Book', 8)

                elif row_number == 2 and column_number >0:
                    value = df['Gross Margin %'][column_number-1] *100
                    cell.text= str(format_number(value,use_decimals= True, decimals=0)) + '%'
                    set_cell_font(cell, 'Nexa Book', 8)

                elif row_number == 3 and column_number >0:
                    value = df['Trade Margin %'][column_number-1]*100
                    cell.text= str(format_number(value,use_decimals= True, decimals=0)) +'%'
                    set_cell_font(cell, 'Nexa Book', 8)
        slide_num +=1            

#### Slide 7:ProductSpendingPool

In [31]:
def productSpendingPool(prs, data,duplication, position):
    slide_num =0
    for key, df in data.items():
        slide = prs.slides[slide_num+position]
        tables, charts = createTableAndChart(slide.shapes)
        table = tables[0].table
        chart = charts[0].chart
        shapes = slide.shapes
        titleNumber = get_shape_number(shapes, "Product Spending pool split | Chocolate | Panda | Hershey | P12M")
        headereNumber = get_shape_number(shapes, "Product Spending pool split (Replace with So What)")
        sourceNumber = get_shape_number(shapes, "Data Source l Client P&L")
        shapes[sourceNumber].text = data_source
        shapes[titleNumber].text = shapes[titleNumber].text.replace('Chocolate', key.split(' | ')[0]).replace('Panda',key.split(' | ')[1]).replace("Hershey",key.split(' | ')[2])
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
        shapes[headereNumber].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[headereNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'


        categories = df['Av Price/Unit'].tolist()
        labeled_categories = [f"Av Price/Unit {format_number(price,use_decimals=True,decimals=2)}" for price in categories]
        chart_data = CategoryChartData()
        chart_data.categories = labeled_categories
        chart_data.add_series('GP', df['Gross Profit/Unit'])
        chart_data.add_series('COGS', df['Total COGS/Unit']*-1)
        chart_data.add_series('TP', df['Trade Profit/Unit'])
        chart_data.add_series('VAT', df['VAT/Unit'])

        chart.replace_data(chart_data)
        num_columns_to_remove = len(table.columns) - df[f'{prodORitem}'].nunique() -1
        table = col_cell_remove(table, num_columns_to_remove)
        for row_number, row in enumerate(table.rows):
            for column_number, cell in enumerate(row.cells):
                if row_number ==0 and column_number >0:
                    value = df[f'{prodORitem}'][column_number-1]
                    cell.text = value
                    set_cell_font(cell, 'Nexa Bold', 8)
                    
                elif row_number == 1 and column_number >0:
                    value = df['Trade Profit/Unit'][column_number-1] / df['Gross Profit/Unit'][column_number-1]
                    cell.text= str(format_number(value,use_decimals= True,decimals=1))
                    set_cell_font(cell, 'Nexa Book', 8)

                elif row_number == 2 and column_number >0:
                    value = df['Gross Margin %'][column_number-1] *100
                    cell.text= str(format_number(value,use_decimals= True, decimals=0)) + '%'
                    set_cell_font(cell, 'Nexa Book', 8)

                elif row_number == 3 and column_number >0:
                    value = df['Trade Margin %'][column_number-1]*100
                    cell.text= str(format_number(value,use_decimals= True, decimals=0)) +'%'
                    set_cell_font(cell, 'Nexa Book', 8)
        slide_num +=1            

#### Slide 8:SKUProfitability

In [32]:
def SKUProfitability(prs,dataframe,numOfDuplicates,position=0):
    for slidenum in range(numOfDuplicates):
        market=list(dataframe.keys())[slidenum]
        df=dataframe[market].copy()
        shapes = prs.slides[slidenum+position].shapes
        charts = []
        tables = []
        title = shapes.title.text
        #for brand in df['Top Brands'].unique():
        uniq_brand_df = df#[df['Top Brands'] == brand]

        # headerNumber = get_shape_number(shapes, "SKU Profitability Analysis with TM% (Replace with So What)")
        titleNumber = 5
        # titleNumber = get_shape_number(shapes, "Gross Margin %, Trade Margin % and Product Sales Rate | Chocolate | Extra | Freia | P12M\nBubble Size: Product Sales Rate")
        #shapes[titleNumber].text = 'Gross Margin %, Trade Margin % and Product Sales Rate  | '+market+' | '+brand+ ' | P12M''\n''Bubble Size: Product Sales Rate'
        shapes[titleNumber].text = 'Gross Margin %, Trade Margin % and Product Sales Rate  | '+market +' | P12M''\n''Bubble Size: Product Sales Rate'

        
        shapes[titleNumber].text_frame.paragraphs[0].font.bold = False
        shapes[titleNumber-1].text = data_source
        
        for shape in shapes:
            if shape.has_chart:
                shape_id = shape.shape_id
                charts.append(shape)
        chart = charts[0].chart
        # charts[0].left = Inches(0.57)
        chart_name = charts[0].name
        chart_type = chart.chart_type
        chart_data = BubbleChartData()
        chart_data.categories = uniq_brand_df['Trade Margin %'].unique().tolist()
        series = chart_data.add_series("Gross Margin %")
        series.has_data_labels = True
        
        for i in range(uniq_brand_df.shape[0]):
            series.add_data_point(uniq_brand_df['Trade Margin %'].iloc[i], uniq_brand_df['Gross Margin %'].iloc[i], uniq_brand_df[f'{prodORitem} Sales Rate'].iloc[i])
        chart.replace_data(chart_data)
        xlsx_file=BytesIO()
        with chart_data._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
            chart_data._workbook_writer._populate_worksheet(workbook, worksheet)
            worksheet.write(0, 4, "labels")
            worksheet.write_column(1, 4, uniq_brand_df[f'{prodORitem}'], None)

        chart._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

        category_axis = chart.category_axis
        if sign == 'Before':
            category_axis.tick_labels.number_format = f'"{currency}"#,##0.00'  if decimals == 2 else f'"{currency}"#,##0'
        else:
            category_axis.tick_labels.number_format = f'#,##0.00"{currency}"'  if decimals == 2 else f'#,##0"{currency}"'
            
        category_axis.auto_axis = True
        
        value_axis = chart.value_axis
        value_axis.tick_labels.number_format = '0%'
        value_axis.auto_axis = True
        

#### Slide 9:TradeMarginTable

In [ ]:
def TradeMarginTable(prs, dataframe_client,dataframe_competitor, numOfDuplicates, position=0,Inscope=""):
    # Loop through each slide number
    # fields_list = {"Sector","Volume Sales","Av Price/KG","Trade Profit","Gross Profit","TP_GP_ratio","Trade Profit/Kg","Gross Margin %","Trade Margin %","Top Companies" }
    
    for slidenum in range(numOfDuplicates):
        # Get market from dataframe keys
        market = list(dataframe_client.keys())[slidenum]
        # Retrieve DataFrame for the current market
        df= dataframe_client[market].reset_index(drop=True)
        dc=dataframe_competitor[market].reset_index(drop=True)
        # Update title
        TP_GP_ratio_client=(df['Trade Profit']/df['Gross Profit']).fillna(0)
        TP_GP_ratio_Comp=(dc['Trade Profit']/dc['Gross Profit']).fillna(0)
        shapes = prs.slides[slidenum + position].shapes

        headerNumber = get_shape_number(shapes, "Trade margin table vs Competition (Replace with So What) ")
        titleNumber = headerNumber-1
        shapes[titleNumber].text = 'Trade margin table vs Competition | '+market+' | P12M'
        shapes[titleNumber-1].text = data_source
    
        # Create table and chart
        tables, charts = createTableAndChart(shapes)
        table = tables[0].table
        # Remove unnecessary rows
        num_rows_to_remove = len(table.rows) - df[Inscope].nunique() -2
        table_height = get_table_height(table)
        for _ in range(num_rows_to_remove):
            if len(table.rows) > 1:
                row = table.rows[1]
        table = removeRowFromTable(table, num_rows_to_remove, 2 )
        # Adjust row heights
        total_row_height = table_height - table.rows[0].height
        num_rows = len(table.rows) - 1
        if num_rows > 0:
            cell_height = total_row_height / num_rows
            for row in range(2, len(table.rows)):
                table.rows[row].height = int(cell_height)
        top_com=dc[f'{ManufOrTopC}'].unique()
        # Populate table cells
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if i==0:
                    if j==1:
                        cell.text="Customer Profit from "+f'{client_manuf[0]}'
                    if j==9:
                        cell.text="Customer Profit from "+f'{top_com[0]}'
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                    cell.text_frame.paragraphs[0].font.size = Pt(10)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)

                if i<2 :
                    continue
                 #####Client Table 
                else:    
                    if j == 0:  # inscope column
                        cell.text = df[Inscope].iloc[i - 2]
                    elif j == 1 and 'Volume Sales' in df.columns:  # Volume Sales column
                        cell.text = f'{float(round(df["Volume Sales"].fillna(0).iloc[i - 2]/percent,1))}'  # Adjusted indexing
                    elif j == 2:  # Av price/kg column
                        cell.text = str(float(round(df['Av Price/KG'].fillna(0).iloc[i - 2], 1)))  # Adjusted indexing
                    elif j == 3 and 'Trade Profit' in df.columns:  # Trade Profit column
                        cell.text = f"{float(round(df['Trade Profit'].fillna(0).iloc[i - 2] /percent,1))}"  # Adjusted indexing
                    elif j == 4 and 'Gross Profit' in df.columns:  # Gross Profit column
                        cell.text = f"{float(round(df['Gross Profit'].fillna(0).iloc[i - 2] /percent,1))}"  # Adjusted indexing
                    elif j == 5:  # TP_GP_ratio column
                        cell.text = f"{TP_GP_ratio_client.iloc[i - 2]:.2f}"  # Display TP_GP_ratio with two decimal places
                    elif j == 6 and 'Trade Profit/Kg' in df.columns:  # Trade Profit/kg column
                        cell.text = f"{float(round(df['Trade Profit/Kg'].fillna(0).iloc[i - 2] , 1))}"  # Adjusted indexing
                    elif j == 7 and 'Gross Margin %' in df.columns:  # Trade Profit/kg column
                        cell.text = f"{float(round(df['Gross Margin %'].fillna(0).iloc[i - 2]*100 , 1))}%"  # Adjusted indexing
                    elif j == 8 and 'Trade Margin %' in df.columns:  # Trade Profit/kg column
                        cell.text = f"{float(round(df['Trade Margin %'].fillna(0).iloc[i - 2]*100 , 1))}%"  # Adjusted indexing
                        ######competitor Table
                    elif j == 9 and 'Volume Sales' in dc.columns:  # Volume Sales column
                        cell.text = f'{float(round(dc["Volume Sales"].fillna(0).iloc[i - 2]/percent,1))}'  # Adjusted indexing
                    elif j == 10:  # Av price/kg column
                        cell.text = str(float(round(dc['Av Price/KG'].fillna(0).iloc[i - 2], 1)))  # Adjusted indexing
                    elif j == 11 and 'Trade Profit' in dc.columns:  # Trade Profit column
                        cell.text = f"{float(round(dc['Trade Profit'].fillna(0).iloc[i - 2] /percent,1))}"  # Adjusted indexing
                    elif j == 12 and 'Gross Profit' in dc.columns:  # Gross Profit column
                        cell.text = f"{float(round(dc['Gross Profit'].fillna(0).iloc[i - 2] /percent))}"  # Adjusted indexing
                    elif j == 13:  # TP_GP_ratio column
                        cell.text = f"{TP_GP_ratio_Comp.iloc[i - 2]:.2f}"  # Display TP_GP_ratio with two decimal places
                    elif j == 14 and 'Trade Profit/Kg' in dc.columns:  # Trade Profit/kg column
                        cell.text = f"{float(round(dc['Trade Profit/Kg'].fillna(0).iloc[i - 2] , 1))}"  # Adjusted indexing
                    elif j == 15 and 'Gross Margin %' in dc.columns:  # Trade Profit/kg column
                        cell.text = f"{float(round(dc['Gross Margin %'].fillna(0).iloc[i - 2]*100 , 1))}%"  # Adjusted indexing
                    elif j == 16 and 'Trade Margin %' in dc.columns:  # Trade Profit/kg column
                        cell.text = f"{float(round(dc['Trade Margin %'].fillna(0).iloc[i - 2]*100 , 1))}%"  # Adjusted indexing
                
                    cell.text_frame.paragraphs[0].font.name = 'Nexa Book'
                    cell.text_frame.paragraphs[0].font.size = Pt(8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER